# Datasets & DataLoaders

We want our dataset code to be separate from our model training code such that modularity and readability of the code enchances. PyTorch provides two data primitives: 
-   torch.utils.data.DataLoader
-   torch.utils.data.Dataset 

that allows us to use pre-loaded datasets as well as your own data. 

    Dataset stores the samples and their corresponding labels
    DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

PyTorch library provide a number of pre-loaded datasets (such as FashionMNIST) that are subcalss of torch.utils.data.Dataset and implement functions specific to the particular data. 

They can be used to prototype and benchmark models. It can be found here: 
*   [Image Datasets](https://pytorch.org/vision/stable/datasets.html)
*   [Text Datasets](https://pytorch.org/text/stable/datasets.html)
*   [Audio Datasets](https://pytorch.org/audio/stable/datasets.html)

## Loading a dataset

We will be using Fashion-MNIST dataset for demonstrating how to load the data from TorchVision. 

Fashion-MNIST is a dataset of Zalando’s article images consisting of 60,000 training examples and 10,000 test examples. Each example comprises a 28×28 grayscale image and an associated label from one of 10 classes.

We load the FashionMNIST Dataset with the following parameters:

*   <b>root</b> is the path where the train/test data is stored,
*   <b>train</b> specifies training or test dataset,
*   <b>download=True</b> downloads the data from the internet if it’s not available at root.
*   <b>transform and target_transform</b> specify the feature and label transformations

In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
%matplotlib inline

# train=true
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# test=true
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

## Iterating and Visualizing the Dataset

We can index Datasets manually like a list: <b>training_data[index]</b>. 

We use matplotlib to visualize some samples in our training data.

In [3]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

# sets the size of the figure
figure = plt.figure(figsize=(8, 8))

# sets the number of rows and columns for sub plots of the figure
cols, rows = 3, 3

# iterate over to all sub plots to plot the fugure.
# for i in range(1, cols * rows + 1):
#     sample_idx = torch.randint(len(training_data), size=(1,)).item() # randonly pick an integer from the length of the training data and use it as index.
#     img, label = training_data[sample_idx] # index the training sample based on choosen index.
#     figure.add_subplot(rows, cols, i) # set the plotting index in subplots matrix in the figure.
#     plt.title(labels_map[label]) # set the title of the plot
#     plt.axis("off") 
#     plt.imshow(img.squeeze(), cmap="gray") 
# plt.show()

<Figure size 576x576 with 0 Axes>

## Creating a Custom Dataset for your files

A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.


In [4]:
import pandas as pd
import os 
from torchvision.io import read_image

In [5]:
class CustomImageDatset(Dataset):
    
    def __init__(self, annotation_file, image_dir, transform=None, target_transform=None):
        """
        The __init__ function is run once when instantiating the Dataset object. 
        We initialize the directory containing the images, the annotations file, and both transforms.
        """
        self.image_labels=pd.read_csv(annotation_file)
        self.img_dir=image_dir
        self.transform=transform
        self.target_transform=target_transform

    def __len__(self):\
    
        """
        The __len__ function returns the number of samples in our dataset.
        """
        return len(self.image_labels)
    
    def __getitem__(self,idx):
        """
        The __getitem__ function loads and returns a sample from the dataset at the given index idx. 
        Based on the index, it identifies the image’s location on disk, converts that to a tensor using read_image.
        Retrieves the corresponding label from the csv data in self.img_labels, calls the transform functions on them (if applicable), 
        and returns the tensor image and corresponding label in a tuple.
        """
        img_path=os.path.join(self.img_dir,self.image_labels.iloc[idx,0])
        image=read_image(img_path)
        label=self.image_labels.iloc[idx,1]
        if self.transform:
            image=self.transform(image)
        if self.target_transform:
            label=self.target_transform(label)
        return image, label






## Preparing your data for training with DataLoaders

The Dataset retrieves our dataset’s features and labels one sample at a time. 

While training a model, we typically want-
*   to pass samples in “minibatches”, 
*   reshuffle the data at every epoch to reduce model overfitting and 
* use Python’s multiprocessing to speed up data retrieval.

<b>DataLoader</b> is an iterable that abstracts this complexity for us in an easy API.

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

We have loaded that dataset into the DataLoader and can iterate through the dataset as needed. 

Each iteration below returns a batch of train_features and train_labels (containing batch_size=64 features and labels respectively).

Because we specified shuffle=True, after we iterate over all batches the data is shuffled.

In [7]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
print(f"Label: {label}")

Feature batch shape: torch.Size([64, 1, 28, 28])
Labels batch shape: torch.Size([64])
Label: 6


Further reading : https://pytorch.org/docs/stable/data.html